<a href="https://colab.research.google.com/github/activeloopai/examples/blob/main/colabs/Creating_Datasets_with_Sequences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Creating Datasets with Sequences***



#### Deep learning with computer vision is increasingly moving in a direction of temporal data, where video frames and their labels are stored as sequences, rather than independent images. Models trained on this data directly account for the temporal information content, rather than making predictions frame-by-frame and then fusing them with non-deep-learning techniques.

## Install Hub

In [ ]:
from IPython.display import clear_output
!pip3 install hub
clear_output()

## Create the Hub Dataset

The first step is to download the dataset [Multiple Object Tracking Benchmark](https://motchallenge.net/data/MOT16/). Additional information about this data and its format is in this [GitHub Repo](https://github.com/JonathonLuiten/TrackEval/blob/master/docs/MOTChallenge-Official/Readme.md).
The dataset has the following folder structure:

MOT16
- train
    - MOT16_N (Folder with sequence N)
        - det
        - gt (Folder with ground truth annotations)
        - img1 (Folder with images the sequence)
            - 00000n.jpg (image of n-th frame in sequence)
    - MOT16_M
    - ...
- test (same structure as train)

In [ ]:
# Download dataset
from IPython.display import clear_output
!wget https://motchallenge.net/data/MOT16.zip
clear_output()

In [ ]:
# Unzip to './MOT16' folder
!unzip -qq /content/MOT16.zip -d "/content/MOT16"

The annotations in `gt.txt` have the format below, and the last 4 items (conf->z) are not used in the hub dataset:

```
frame, id, bb_left, bb_top, bb_width, bb_height, conf, x, y, z
```



Now we're ready to **create a Hub Dataset** in the `./mot_2016_train` folder by running:

In [ ]:
%env BUGGER_OFF = True
import hub
import os
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw

ds = hub.empty('./mot_2016_train', overwrite = True) # Create the dataset locally

env: BUGGER_OFF=True
NumExpr defaulting to 2 threads.


Next, let's write code to inspect the folder structure for the downloaded dataset and create a list of folders containing the sequences:

In [ ]:
dataset_folder = '/content/MOT16/train'

sequences = [ item for item in sorted(os.listdir(dataset_folder)) if os.path.isdir(os.path.join(dataset_folder, item)) ]

Next, let's create the tensors by using the `sequence[...]` htype, iterate through each sequence, and iterate through each frame within the sequence, one-by-one. 

**Note:** Data is appended to sequence[...] htypes using lists. The list contains the whole sample, and the individual elements of the list are the individual data points, such as the image frame, the bounding boxes in a particular frame, etc. 

See end of code block below.

In [ ]:
with ds:
    # Define tensors
    ds.create_tensor('frames', htype = 'sequence[image]', sample_compression = 'jpg')
    ds.create_tensor('boxes', htype = 'sequence[bbox]')
    ds.create_tensor('ids', htype = 'sequence[]', dtype = 'uint32') # Ids are not uploaded as htype = 'class_labels' because they don't contain information about the class of an object.

    ds.boxes.info.update(coords = {'type': 'pixel', 'mode': 'LTWH'}) # Bounding box format is left, top, width, height

    # Iterate through each sequence
    for sequence in sequences:

        # Define root directory for that sequence    
        root_local = os.path.join(dataset_folder,sequence, 'img1')
        
        # Get a list of all the image paths
        img_paths = [os.path.join(root_local, item) for item in sorted(os.listdir(root_local))]

        # Read the annotations and convert to dataframe
        with open(os.path.join(dataset_folder,sequence, 'gt', 'gt.txt')) as f:
            anns = [line.rstrip('\n') for line in f]
        
        anns_df = pd.read_csv(os.path.join(dataset_folder, sequence, 'gt', 'gt.txt'), header = None)

        # Get the frames from the annotations and make sure they're of equal length as the images
        frames = pd.unique(anns_df[0])
        assert len(frames) == len(img_paths)

        # Iterate through each frame and add data to sequence
        boxes_seq = []
        ids_seq = []
        for frame in frames:
            ann_df = anns_df[anns_df[0] == frame] # Find annotations in the specific frame

            boxes_seq.append(ann_df.loc[:, [2, 3, 4, 5]].to_numpy().astype('float32')) # Box coordinates are in the 3rd-6th column

            ids_seq.append(ann_df.loc[:, 1].to_numpy().astype('uint32')) # ids are in the second column
        
        # Append the sequences to the hub dataset
        ds.append({
            "frames": [hub.read(path) for path in img_paths],
            "boxes": boxes_seq,
            "ids": ids_seq})

This dataset identifies objects by `id`, where each `id` represents an instance of an object. However, the `id` does not identify the class of the object, such person, car, truck, etc. Therefore, the `ids` were not uploaded as `htype = "class_label"`.

##Inspect the Hub Dataset

Let's check out the 10th frame in the 6th sequence in this dataset.

In [ ]:
# Draw bounding boxes for the 10th frame in the 6th sequence

seq_ind = 5
frame_ind = 9
img = Image.fromarray(ds.frames[seq_ind][frame_ind].numpy())
draw = ImageDraw.Draw(img)
(w,h) = img.size
boxes = ds.boxes[seq_ind][frame_ind].numpy()

for b in range(boxes.shape[0]):
    (x1,y1) = (int(boxes[b][0]), int(boxes[b][1]))
    (x2,y2) = (int(boxes[b][0]+boxes[b][2]), int(boxes[b][1]+boxes[b][3]))
    draw.rectangle([x1,y1,x2,y2], width=2, outline = 'red')

In [ ]:
# Display the frame and its bounding boxes
img

Congrats! You just created a dataset using sequences! 🎉